<a href="https://colab.research.google.com/github/vaishnavieduvarikuti-cpu/Brain-Tumor-ViT-Comparison/blob/main/brain%20tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload() # Upload your kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset -p /content/dataset --unzip

In [ ]:
# ===================================================================
# FINAL CORRECTED ALL-IN-ONE CODE BLOCK
# ===================================================================

# --- CELL 1: IMPORTS ---
import os
import torch
import timm
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm

print("--- Step 1: Libraries Imported ---")

# --- CELL 2: CONFIGURATION ---
DATA_DIR = '/content/dataset/Training/'
TEST_DIR = '/content/dataset/Testing/'
IMAGE_SIZE = 224
BATCH_SIZE = 8 # <<< FINAL CHANGE TO FIX MEMORY ERROR
EPOCHS = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"--- Step 2: Configuration Set (Using device: {device}, Batch Size: {BATCH_SIZE}) ---")

# --- CELL 3: DATA LOADING ---
def load_data(data_dir):
    image_paths, labels = [], []
    class_names = sorted(os.listdir(data_dir))
    label_map = {name: i for i, name in enumerate(class_names)}
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(class_dir, filename))
                labels.append(label_map[class_name])
    return pd.DataFrame({'filepath': image_paths, 'label': labels}), label_map

train_val_df, label_map = load_data(DATA_DIR)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42, stratify=train_val_df['label'])
print("--- Step 3: Data Loaded and Split ---")

# --- CELL 4: PREPROCESSING & DATASETS ---
IMAGENET_MEAN = [0.485, 0.456, 0.46]
IMAGENET_STD = [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(), transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(), transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
val_test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
class TumorDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df, self.transform = dataframe, transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_path, label = self.df.iloc[idx]['filepath'], self.df.iloc[idx]['label']
        image = Image.open(img_path).convert("RGB")
        if self.transform: image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)
train_dataset = TumorDataset(train_df, transform=train_transform)
val_dataset = TumorDataset(val_df, transform=val_test_transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
print("--- Step 4: DataLoaders Ready ---")

# --- CELL 5: TRAINING FUNCTION ---
def train_model(model, model_name, train_loader, val_loader, epochs, learning_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    best_val_accuracy = 0.0
    for epoch in range(epochs):
        model.train()
        print(f"\nEpoch {epoch+1}/{epochs}")
        for images, labels in tqdm(train_loader, desc="Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        model.eval()
        correct_val = 0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                correct_val += (predicted == labels).sum().item()
        val_accuracy = correct_val / len(val_loader.dataset)
        print(f"Validation Accuracy: {val_accuracy:.4f}")
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), f'/content/{model_name}_best.pth')
            print(f"----> New best model saved")
    print(f"\nFinished Training for {model_name}!")
    return best_val_accuracy
print("--- Step 5: Training Function Defined ---")

# --- FINAL STEP: TRAIN THE TINY MAXVIT MODEL ---
print("\n--- Starting Final Step: Training the Tiny MaxViT Model ---")
maxvit_model_name = 'maxvit_tiny_tf_224'
maxvit_model = timm.create_model(maxvit_model_name, pretrained=True, num_classes=len(label_map))
maxvit_model.to(device)
maxvit_accuracy = train_model(maxvit_model, maxvit_model_name, train_loader, val_loader, EPOCHS, 1e-5)
print(f"\nBest Validation Accuracy for Tiny MaxViT: {maxvit_accuracy:.4f}")

In [ ]:
# --- FINAL STEP: TRAIN THE TINY SWIN TRANSFORMER ---
print("\n--- Starting Final Step: Training the Tiny Swin Transformer ---")
# NOTE: We are using the 'tiny' Swin model for a fair comparison and to avoid memory errors
swin_model_name = 'swin_tiny_patch4_window7_224'

swin_model = timm.create_model(swin_model_name, pretrained=True, num_classes=len(label_map))
swin_model.to(device)

swin_accuracy = train_model(
    model=swin_model,
    model_name=swin_model_name,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=EPOCHS,
    learning_rate=1e-5
)

print(f"\nBest Validation Accuracy for Tiny Swin Transformer: {swin_accuracy:.4f}")

In [ ]:
from google.colab import files
print("Please upload your kaggle.json API token.")
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset -p /content/dataset --unzip

In [ ]:
# Imports
import os
import torch
import timm
import pandas as pd
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

# Configuration
DATA_DIR = '/content/dataset/Training/'
TEST_DIR = '/content/dataset/Testing/'
IMAGE_SIZE = 224
BATCH_SIZE = 8 # Use the same batch size as your final successful training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Libraries imported and configuration set. Using device: {device}")

In [ ]:
# Data Loading Function
def load_data(data_dir):
    image_paths, labels = [], []
    class_names = sorted(os.listdir(data_dir))
    label_map = {name: i for i, name in enumerate(class_names)}
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(class_dir, filename))
                labels.append(label_map[class_name])
    return pd.DataFrame({'filepath': image_paths, 'label': labels}), label_map

# Load dataframes
train_val_df, label_map = load_data(DATA_DIR)
test_df, _ = load_data(TEST_DIR)

# Define Transformation
val_test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset Class
class TumorDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.df, self.transform = dataframe, transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_path, label = self.df.iloc[idx]['filepath'], self.df.iloc[idx]['label']
        image = Image.open(img_path).convert("RGB")
        if self.transform: image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

# Create the Test DataLoader
test_dataset = TumorDataset(test_df, transform=val_test_transform)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print("Test data is loaded and ready for evaluation.")

In [ ]:
def evaluate_model(model_name, test_loader, label_map):
    print(f"\n======================================================")
    print(f"   Starting Evaluation for: {model_name}")
    print(f"======================================================")

    model = timm.create_model(model_name, pretrained=False, num_classes=len(label_map))
    model_path = f'/content/{model_name}_best.pth'
    try:
        model.load_state_dict(torch.load(model_path))
    except FileNotFoundError:
        print(f"ERROR: Model file not found at {model_path}.")
        print("Please make sure you have run the training for this model OR that the saved .pth file has been uploaded to /content/.")
        return

    model.to(device)
    model.eval()

    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc=f"Evaluating {model_name}"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    class_names = list(label_map.keys())
    print(f"\n--- Evaluation Report for {model_name} ---")
    print("Accuracy:", accuracy_score(all_preds, all_labels))
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label'); plt.ylabel('True Label')
    plt.title(f'Confusion Matrix for {model_name}'); plt.show()

In [ ]:
# --- This is the code for YOUR project (Swin + MaxViT) ---
print("--- Running Final Evaluation for Srivaishnavi's Project ---")

# Define the EXACT model names YOU trained
swin_model_name = 'swin_tiny_patch4_window7_224'
maxvit_model_name = 'maxvit_tiny_tf_224'

# Call the function for each of your models
# IMPORTANT: Make sure the files 'swin_tiny..._best.pth' and 'maxvit_tiny..._best.pth' are in your /content/ directory.
# If they are not there, you must re-run the training for them first.
evaluate_model(swin_model_name, test_loader, label_map)
evaluate_model(maxvit_model_name, test_loader, label_map)

In [ ]:
from google.colab import files
files.upload() # Upload your kaggle.json

In [ ]:
!mkdir -p ~/.kaggle && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d masoudnickparvar/brain-tumor-mri-dataset -p /content/dataset --unzip

In [ ]:
import os
import torch
import timm
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Add this line for confirmation
print(f"Libraries imported successfully. Using device: {device}")

In [ ]:
# ===================================================================
# ALL-IN-ONE CODE BLOCK FOR EXPERIMENT 1
# ===================================================================

# --- STEP 1: IMPORTS ---
import os
import torch
import timm
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from PIL import Image
from tqdm import tqdm
print("--- Step 1: Libraries Imported ---")

# --- STEP 2: CONFIGURATION ---
DATA_DIR = '/content/dataset/Training/'
IMAGE_SIZE = 224
BATCH_SIZE = 8      # Using the safe batch size
EPOCHS = 35         # <<< EXPERIMENT 1: Training for 35 epochs
LEARNING_RATE = 1e-5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"--- Step 2: Configuration Set (Device: {device}, Epochs: {EPOCHS}) ---")

# --- STEP 3: DATA LOADING ---
def load_data(data_dir):
    image_paths, labels = [], []
    class_names = sorted(os.listdir(data_dir))
    label_map = {name: i for i, name in enumerate(class_names)}
    for class_name in class_names:
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image_paths.append(os.path.join(class_dir, filename))
                labels.append(label_map[class_name])
    return pd.DataFrame({'filepath': image_paths, 'label': labels}), label_map

train_val_df, label_map = load_data(DATA_DIR)
train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42, stratify=train_val_df['label'])
print("--- Step 3: Data Loaded and Split ---")
print(f"Label map: {label_map}")
print(f"Training samples: {len(train_df)}, Validation samples: {len(val_df)}")

# --- STEP 4: PREPROCESSING & DATASETS ---
IMAGENET_MEAN, IMAGENET_STD = [0.485, 0.456, 0.406], [0.229, 0.224, 0.225]
train_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(), transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(), transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
val_test_transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)), transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])
class TumorDataset(Dataset):
    def __init__(self, df, transform=None): self.df, self.transform = df, transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        img_path, label = self.df.iloc[idx]['filepath'], self.df.iloc[idx]['label']
        image = Image.open(img_path).convert("RGB")
        if self.transform: image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)
train_dataset, val_dataset = TumorDataset(train_df, transform=train_transform), TumorDataset(val_df, transform=val_test_transform)
train_loader, val_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2), DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
print("--- Step 4: DataLoaders Ready ---")

# --- STEP 5: TRAINING FUNCTION ---
def train_model(model, model_name, train_loader, val_loader, epochs, learning_rate):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
    best_val_accuracy = 0.0
    for epoch in range(epochs):
        model.train()
        print(f"\nEpoch {epoch+1}/{epochs}")
        for images, labels in tqdm(train_loader, desc="Training"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        model.eval()
        correct_val = 0
        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc="Validation"):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                correct_val += (predicted == labels).sum().item()
        val_accuracy = correct_val / len(val_loader.dataset)
        print(f"Validation Accuracy: {val_accuracy:.4f}")
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), f'/content/{model_name}_best.pth')
            print(f"----> New best model saved")
    print(f"\nFinished Training for {model_name}!")
    return best_val_accuracy
print("--- Step 5: Training Function Defined ---")

# --- FINAL STEP: TRAIN THE TINY MAXVIT MODEL ---
print("\n--- Starting Final Step: Training the Tiny MaxViT Model ---")
maxvit_model_name = 'maxvit_tiny_tf_224'
maxvit_model = timm.create_model(maxvit_model_name, pretrained=True, num_classes=len(label_map))
maxvit_model.to(device)
maxvit_accuracy = train_model(maxvit_model, maxvit_model_name, train_loader, val_loader, EPOCHS, LEARNING_RATE)
print(f"\nBest Validation Accuracy for Tiny MaxViT (Experiment 1): {maxvit_accuracy:.4f}")

In [ ]:
# --- This is the code for YOUR project (Swin + MaxViT) ---
print("--- Running Final Evaluation for Srivaishnavi's Project ---")

# Define the EXACT model names YOU trained
swin_model_name = 'swin_tiny_patch4_window7_224'
maxvit_model_name = 'maxvit_tiny_tf_224'

# Call the function for each of your models
# IMPORTANT: Make sure the files 'swin_tiny..._best.pth' and 'maxvit_tiny..._best.pth' are in your /content/ directory.
# If they are not there, you must re-run the training for them first.
evaluate_model(swin_model_name, test_loader, label_map)
evaluate_model(maxvit_model_name, test_loader, label_map)